In [ ]:
%create_livy_session \
--cluster dp-dl \
--id ses1 \
--conf spark.jars.packages=io.delta:delta-core_2.12:0.8.0 \
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension \
--conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog \
--conf spark.sql.hive.metastore.sharedPrefixes=com.amazonaws,ru.yandex.cloud \
--conf spark.sql.warehouse.dir=s3a://keshaaa/wh

In [25]:
#!spark --cluster dp-dl --session ses1
df = spark.read.option("delimiter", "\t").option("header", False).csv("s3a://keshaaa/alfastrah-users.csv")
new_names = ['id', 'date', 'email-1', 'password', 'FirstName', 'LastName', 'e-mail-2', 'date-2', 'phone', 'birthdate', 'MiddleName']
df = df.toDF(*new_names)
df.show()

+--------+-------------------+--------------------+--------------------+---------+-------------+--------------------+-------------------+-----------+----------+-------------+
|      id|               date|             email-1|            password|FirstName|     LastName|            e-mail-2|             date-2|      phone| birthdate|   MiddleName|
+--------+-------------------+--------------------+--------------------+---------+-------------+--------------------+-------------------+-----------+----------+-------------+
|13541159|2023-02-12 00:43:25|  M.PISAREVA@MAIL.RU|3fE_A6+nc44fa1e3a...|   Марина|     Писарева|  M.PISAREVA@MAIL.RU|2023-02-12 00:43:25|79219249780|1981-12-05|   Викторовна|
|13541160|2022-09-17 06:21:37|   Mail30rus@mail.ru|tQNlQNMAbf7ab376f...|     Иван|        Жуков|   Mail30rus@mail.ru|2022-09-17 06:21:37|79272847965|1990-07-31|  Анатольевич|
|13541161|2022-09-09 12:34:35|order-215619298@i...|s8D4SspZ9e033fedb...|     ИВАН|      СТРУНИН|order-215619298@i...|        

In [16]:
#!spark --cluster dp-dl --session ses1
df.createOrReplaceTempView("table")

In [35]:
#!spark --cluster dp-dl --session ses1
sqlDF = spark.sql("SELECT * FROM table WHERE _c5 = 'Дар'")
sqlDF.show()

+--------+-------------------+-----------------+--------------------+-----+---+-----------------+-------------------+-----------+----------+-------------+
|     _c0|                _c1|              _c2|                 _c3|  _c4|_c5|              _c6|                _c7|        _c8|       _c9|         _c10|
+--------+-------------------+-----------------+--------------------+-----+---+-----------------+-------------------+-----------+----------+-------------+
|13724057|2023-01-28 17:33:27|d.m.dar@yandex.ru|SUNMVv0C1662fdf1a...|Денис|Дар|d.m.dar@yandex.ru|2023-01-28 17:33:27|79167722772|1983-05-18|     Маркович|
|13776485|2023-03-10 17:31:51|      Izeda@ya.ru|O7^oCk9}e8187bccd...|Мария|Дар|      Izeda@ya.ru|2023-03-10 17:31:51|79028007608|1988-05-04|Александровна|
+--------+-------------------+-----------------+--------------------+-----+---+-----------------+-------------------+-----------+----------+-------------+


In [30]:
#!spark --cluster dp-dl --session ses1
df.write.format("delta").mode("overwrite").option(
    "overwriteSchema", "true"
).save(f"s3a://keshaaa/test/delta/")

In [44]:
#!spark --cluster dp-dl --session ses1
spark.sql("USE testdelta;")
spark.sql("CREATE TABLE tab2 USING DELTA AS SELECT * FROM table")

DataFrame[]


In [46]:
#!spark --cluster dp-dl --session ses1
spark.sql("DROP TABLE tab4")

DataFrame[]


In [ ]:
%delete_livy_session \
--cluster dp-dl \
--id ses1